In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from DatasetHandler import DatasetHandler
from findpeaks import findpeaks
import findpeaks
from scipy import special
from tqdm import tqdm

In [ ]:
def enl(img):
    return (np.nanmean(img)**2)/(np.nanstd(img)**2)

In [ ]:
def normalize(s1):
    def reject_outliers_2(data, m=5):
        d = np.abs(data - np.median(data))
        mdev = np.median(d)
        s = d / (mdev if mdev else 1.)
        return data[s < m]  

    d = reject_outliers_2(s1.flatten(), m=8.)
    s1_n = (s1 -  np.min(d))/(np.max(d) -  np.min(d))
    s1_n = np.clip(s1_n, 0.0, 1.0)

    return s1_n.astype(np.float64)

## Load amplitude data

In [ ]:
import rasterio
# path = 'GRD_data/S1-lat_41_13694930422632_lon_14_755163002014147-2019-09-12.tif'
path = 'GRD_data/subset_0_of_S1A_IW_GRDH_1SDV_20211020T174045_20211020T174110_040206_04C337_6D4A_Orb_Bdr_Thermal_Cal_TC.tif'

with rasterio.open(path) as src:
    vv = src.read(1).astype(np.float64)
    vh = src.read(2).astype(np.float64)

intensity = vh

print(' Max', amplitude.max(), '\n Min', amplitude.min(), '\n Std', amplitude.std(), '\n Mean', amplitude.mean())

### Get Intensity

In [ ]:
print(' Max', intensity.max(), '\n Min', intensity.min(), '\n Std', intensity.std(), '\n Mean', intensity.mean())

### Normalize intensity

In [ ]:
norm_intensity = normalize(intensity)
print(' Max', norm_intensity.max(), '\n Min', norm_intensity.min(), '\n Std', norm_intensity.std(), '\n Mean', norm_intensity.mean())

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize = (10,10))
ax.hist(norm_intensity.flatten(), 300)
ax.set_ylim([0,2e6])
plt.show()

## Plot data

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize = (20,20))
ax.imshow(norm_intensity, cmap ='gray', vmin = 0.1, vmax = 0.8)

#major_x_ticks = np.arange(0, norm_intensity.shape[1], 256)
#major_y_ticks = np.arange(0, norm_intensity.shape[0], 256)
#ax.set_xticks(major_x_ticks)
#ax.set_yticks(major_y_ticks)
#ax.grid(which='both',color='r', linestyle='-', linewidth=2)

ax.axis(False)
plt.show()

# Proposed filter

In [ ]:
import time
from Model import CNNSpeckleFilter
from tensorflow.keras.models import load_model

N_LAYER = 12
IMG_SHAPE = (96,96,1)

speckle_filter = CNNSpeckleFilter(input_shape=IMG_SHAPE, n_layers=N_LAYER)
speckle_filter.model = load_model('weights/new_model_'+str(N_LAYER)+'layers.h5')


In [ ]:
start_time = time.time()

a = speckle_filter.model.predict(norm_intensity[np.newaxis, 0:96, 0:96, np.newaxis])

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
prediction = np.zeros(norm_intensity.shape)

start_time = time.time()

for i in range(norm_intensity.shape[0]//IMG_SHAPE[0]):
    for j in range(norm_intensity.shape[1]//IMG_SHAPE[1]):
        prediction[i*IMG_SHAPE[0]:(i+1)*IMG_SHAPE[0],j*IMG_SHAPE[1]:(j+1)*IMG_SHAPE[1]] = speckle_filter.model.predict(norm_intensity[np.newaxis, i*IMG_SHAPE[0]:(i+1)*IMG_SHAPE[0],j*IMG_SHAPE[1]:(j+1)*IMG_SHAPE[1],np.newaxis])[0,...,0]
        
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize = (20,20))
ax.imshow(prediction, cmap ='gray', vmin = 0.1, vmax=0.8)

#major_x_ticks = np.arange(0, norm_intensity.shape[1], 256)
#major_y_ticks = np.arange(0, norm_intensity.shape[0], 256)
#ax.set_xticks(major_x_ticks)
#ax.set_yticks(major_y_ticks)
#ax.grid(which='both',color='r', linestyle='-', linewidth=2)

ax.axis(False)
plt.show()

# Lee filter

In [ ]:
import findpeaks
scaled_intensity = findpeaks.stats.scale(norm_intensity)
# window size
winsize = 3
# damping factor for frost
k_value1 = 2.0
# damping factor for lee enhanced
k_value2 = 1.0
# coefficient of variation of noise
cu_value = 0.25
# coefficient of variation for lee enhanced of noise
cu_lee_enhanced = 0.523
# max coefficient of variation for lee enhancedW
cmax_value = 1.73

In [ ]:
start_time = time.time()
_ = findpeaks.lee_enhanced_filter(scaled_intensity[:96,:96,...], win_size=winsize, k=k_value2, cu=cu_lee_enhanced, cmax=cmax_value)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
start_time = time.time()
image_lee_enhanced = findpeaks.lee_enhanced_filter(scaled_intensity, win_size=winsize, k=k_value2, cu=cu_lee_enhanced, cmax=cmax_value)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize = (20,20))
ax.imshow(image_lee_enhanced, cmap ='gray', vmin=20, vmax=220)
ax.axis(False)
plt.show()

# SARBM3D

In [ ]:
import rasterio
# path = 'GRD_data/S1-lat_41_13694930422632_lon_14_755163002014147-2019-09-12.tif'
path = 'SARBM3D.tif'

with rasterio.open(path) as src:
    sarbm3d = src.read(1).astype(np.float64)


In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize = (20,20))
ax.imshow(sarbm3d, cmap ='gray', vmin=0, vmax=200)
ax.axis(False)
plt.show()

#print('ENL IDCNN', enl(BM3D[100:150, 0:50]))